In [3]:
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
from itertools import chain

client = MongoClient("mongodb://localhost:27017/")
db = client["recommender_system"]  

In [4]:
df = pd.read_csv('NIES 2024.csv')
df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q13,Q14,Q15,Q16,...,HospSize,admcode,xadmtype,AdmTypeBinary,ScaleADM,ScaleTREAT,ScaleDCHARGE,ScaleOTHER,ScoreWAIT,ScoreOVERALL
0,2.0,NaN,NaN,NaN,NaN,NaN,4.0,1.0,1.0,4.0,...,2.0,2,2,1.0,NaN,9.38,7.14,10.00,NaN,10.0
1,1.0,1.0,4.0,2.0,2.0,6.0,3.0,2.0,2.0,2.0,...,2.0,1,1,0.0,5.00,5.00,5.63,3.33,NaN,3.0
2,1.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,...,2.0,2,1,0.0,NaN,10.00,9.38,10.00,NaN,10.0
3,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,2.0,1,1,0.0,8.33,10.00,10.00,10.00,3.0,10.0
4,2.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,...,2.0,1,2,1.0,NaN,9.38,8.13,10.00,NaN,9.0


In [ ]:
# df.insert(0, 'Response_ID', range(1, len(df) + 1))
# df.to_csv('NIES 2024.csv', index=False)

# Treatment, Procedures, staff interaction

In [9]:
df[['Response_ID','Q13','Q14','Q15','Q16','Q17','Q18','Q19','Q20','Q21','Q22','Q23','Q24','Q25','Q26','Q22']]

,Response_ID,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q22
0,1,4.0,1.0,1.0,4.0,1.0,2.0,1.0,6.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0
1,2,3.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0
2,3,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0,1.0,2.0,1.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0
4,5,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12362,12363,2.0,2.0,3.0,3.0,2.0,1.0,2.0,6.0,3.0,3.0,1.0,1.0,4.0,3.0,3.0
12363,12364,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0
12364,12365,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
12365,12366,2.0,3.0,2.0,1.0,3.0,1.0,2.0,6.0,4.0,2.0,2.0,1.0,2.0,2.0,2.0


Q13		
        Value
Standard Attributes	Position	7
    Label	Answers from doctor you could understand
Valid Values	1	Yes, always
    2	Yes, sometimes
    3	No
    4	Had no need to ask


In [ ]:
# Q13 - ignoring 4 as its not a valid option
# df['Q13'] = df['Q13'].replace(4, None)

In [16]:
question_cols = [f"Q{i}" for i in range(13, 28)]
mapping = {1: 1.0, 2: 0.5, 3: 0.0}
for col in question_cols:
    # df[col] = df[col].replace(4, np.nan)      # Exclude '4'
    df[col] = df[col].map(mapping)            # Map to [0, 1]


In [18]:
df[['Response_ID','Q13','Q14','Q15','Q16','Q17','Q18','Q19','Q20','Q21','Q22','Q23','Q24','Q25','Q26','Q22']]

,Response_ID,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q22
0,1,NaN,1.0,1.0,NaN,1.0,0.5,1.0,NaN,NaN,1.0,0.5,1.0,1.0,1.0,1.0
1,2,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.5,0.5,0.0,0.5,0.5,0.5,0.5
2,3,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,4,1.0,1.0,1.0,1.0,1.0,0.5,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0
4,5,1.0,1.0,1.0,1.0,0.5,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12362,12363,0.5,0.5,0.0,0.0,0.5,1.0,0.5,NaN,0.0,0.0,1.0,1.0,NaN,0.0,0.0
12363,12364,0.5,1.0,1.0,1.0,1.0,0.5,1.0,0.5,0.0,1.0,1.0,1.0,1.0,1.0,1.0
12364,12365,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
12365,12366,0.5,0.0,0.5,1.0,0.0,1.0,0.5,NaN,NaN,0.5,0.5,1.0,0.5,0.5,0.5


In [19]:
df.dropna(subset=question_cols, inplace=True)

In [22]:
df[question_cols]

,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
1,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.5,0.5,0.0,0.5,0.5,0.5,0.5
2,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,0.5,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12349,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
12357,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,0.5
12363,0.5,1.0,1.0,1.0,1.0,0.5,1.0,0.5,0.0,1.0,1.0,1.0,1.0,1.0,1.0
12364,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [23]:
# Treatment, procedures, staff interaction
treatment_cols = [f"Q{i}" for i in range(13, 25)]  # Q13 to Q24
# Pain management
pain_cols = ["Q25"]
# Explanation of risks/outcomes
explanation_cols = ["Q26", "Q27"]

In [24]:
# Calculate mean scores per section
df["treatment_score"] = df[treatment_cols].mean(axis=1, skipna=True)
df["pain_score"] = df[pain_cols].mean(axis=1, skipna=True)
df["explanation_score"] = df[explanation_cols].mean(axis=1, skipna=True) 

In [26]:
df["outcome_proxy"] = df["Q42"] / 10

In [27]:
df[['outcome_proxy']]

,outcome_proxy
1,0.3
2,1.0
4,0.9
5,1.0
8,1.0
...,...
12349,0.8
12357,1.0
12363,1.0
12364,1.0


In [28]:
df["satisfaction_score"] = (
    1 * df["treatment_score"] +
    1 * df["pain_score"] +
    1 * df["explanation_score"] +
    2 * df["outcome_proxy"]
) / 5

In [29]:
df['satisfaction_score']

1        0.395000
2        0.991667
4        0.935000
5        0.991667
8        0.983333
           ...   
12349    0.911667
12357    0.933333
12363    0.958333
12364    0.991667
12366    0.721667
Name: satisfaction_score, Length: 4776, dtype: float64

In [30]:
df

,Response_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q13,Q14,Q15,...,ScaleTREAT,ScaleDCHARGE,ScaleOTHER,ScoreWAIT,ScoreOVERALL,treatment_score,pain_score,explanation_score,outcome_proxy,satisfaction_score
1,2,1.0,1.0,4.0,2.0,2.0,6.0,0.0,0.5,0.5,...,5.00,5.63,3.33,NaN,3.0,0.375000,0.5,0.50,0.3,0.395000
2,3,1.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,10.00,9.38,10.00,NaN,10.0,0.958333,1.0,1.00,1.0,0.991667
4,5,2.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,9.38,8.13,10.00,NaN,9.0,0.875000,1.0,1.00,0.9,0.935000
5,6,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,...,10.00,10.00,10.00,3.0,10.0,0.958333,1.0,1.00,1.0,0.991667
8,9,2.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,9.38,10.00,10.00,NaN,10.0,0.916667,1.0,1.00,1.0,0.983333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12349,12350,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,...,10.00,10.00,10.00,9.0,8.0,0.958333,1.0,1.00,0.8,0.911667
12357,12358,1.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,9.38,10.00,10.00,NaN,10.0,0.916667,1.0,0.75,1.0,0.933333
12363,12364,1.0,1.0,1.0,2.0,2.0,2.0,0.5,1.0,1.0,...,10.00,6.67,8.33,9.0,10.0,0.791667,1.0,1.00,1.0,0.958333
12364,12365,1.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,...,10.00,9.38,10.00,18.0,10.0,0.958333,1.0,1.00,1.0,0.991667


# ICD Mapping begins

In [ ]:
 
condition_mapping = [
    {
        'condition_label': 'Tumour or cancer',
        'icd10_from': 'C00', 'icd10_to': 'D49',
        'icd9_from': '140', 'icd9_to': '239'
    },
    {
        'condition_label': 'Heart condition',
        'icd10_from': 'I00', 'icd10_to': 'I99',
        'icd9_from': '390', 'icd9_to': '459'
    },
    {
        'condition_label': 'Lung condition',
        'icd10_from': 'J00', 'icd10_to': 'J99',
        'icd9_from': '460', 'icd9_to': '519'
    },
    {
        'condition_label': 'Neurological condition',
        'icd10_from': 'G00', 'icd10_to': 'G99',
        'icd9_from': '320', 'icd9_to': '389'
    },
    {
        'condition_label': 'Orthopaedic condition',
        'icd10_from': 'M00', 'icd10_to': 'M99',
        'icd9_from': '710', 'icd9_to': '739'
    },
    {
        'condition_label': 'COVID 19',
        'icd10_from': 'U07', 'icd10_to': 'U07',
        'icd9_from': None, 'icd9_to': None
    },
    {
        'condition_label': 'Infection (other than COVID 19)',
        'icd10_from': 'A00', 'icd10_to': 'B99',
        'icd9_from': '001', 'icd9_to': '139'
    },
    {
        'condition_label': 'Digestive system condition',
        'icd10_from': 'K00', 'icd10_to': 'K95',
        'icd9_from': '520', 'icd9_to': '579'
    },
    {
        'condition_label': 'Diabetes and related problems',
        'icd10_from': 'E08', 'icd10_to': 'E13',
        'icd9_from': '250', 'icd9_to': '250'
    },
    {
        'condition_label': 'Adverse reaction or poisoning',
        'icd10_from': 'T36', 'icd10_to': 'T50',
        'icd9_from': '960', 'icd9_to': '989'
    },
    {
        'condition_label': 'Injury and/or accident',
        'icd10_from': 'S00', 'icd10_to': 'T88',
        'icd9_from': '800', 'icd9_to': '959'
    },
    {
        'condition_label': 'Mental health issue',
        'icd10_from': 'F00', 'icd10_to': 'F99',
        'icd9_from': '290', 'icd9_to': '319'
    },
    {
        'condition_label': 'Tests and/or investigations',
        'icd10_from': 'Z01', 'icd10_to': 'Z01',
        'icd9_from': 'V72', 'icd9_to': 'V72'
    },
    {
        'condition_label': 'Don’t know or wasn’t told',
        'icd10_from': None, 'icd10_to': None,
        'icd9_from': None, 'icd9_to': None
    },
    {
        'condition_label': 'Other',
        'icd10_from': None, 'icd10_to': None,
        'icd9_from': None, 'icd9_to': None
    }
]

# Normalize ICD code: take first 3 alphanum characters
def normalize_icd_code(code):
    code = str(code).upper().replace('.', '')
    return code[:3] if code else None

# Apply mapping function
def map_icd_info(icd_code, icd_version):
    icd_code = normalize_icd_code(icd_code)
    if not icd_code or not icd_version:
        return pd.Series(['Other', None, None])
    
    for entry in condition_mapping:
        if icd_version == 10:
            start, end = entry['icd10_from'], entry['icd10_to']
        elif icd_version == 9:
            start, end = entry['icd9_from'], entry['icd9_to']
        else:
            continue

        if start and end and start <= icd_code <= end:
            return pd.Series([entry['condition_label'], start, end])
    
    return pd.Series(['Other', None, None])

# Assuming your main DataFrame is df and it has columns 'icd_code' and 'icd_version'
df[['condition_label', 'icd_code_range_from', 'icd_code_range_to']] = df.apply(
    lambda row: map_icd_info(row['icd_code'], row['icd_version']), axis=1
)


In [78]:
import pandas as pd

# Sample mapping
mapping = {
    'Tumour or cancer': ('C00', 'D49', '140', '239'),
    'Heart condition': ('I00', 'I99', '390', '459'),
    'Lung condition': ('J00', 'J99', '460', '519'),
    'Neurological condition': ('G00', 'G99', '320', '389'),
    'Orthopaedic condition': ('M00', 'M99', '710', '739'),
    'COVID 19': ('U07', 'U07', None, None),
    'Infection (other than COVID 19)': ('A00', 'B99', '001', '139'),
    'Digestive system condition': ('K00', 'K95', '520', '579'),
    'Diabetes and related problems': ('E08', 'E13', '250', '250'),
    'Adverse reaction or poisoning': ('T36', 'T50', '960', '989'),
    'Injury and/or accident': ('S00', 'T88', '800', '959'),
    'Mental health issue': ('F00', 'F99', '290', '319'),
    'Tests and/or investigations': ('Z01', 'Z01', 'V72', 'V72'),
    'Don’t know or wasn’t told': (None, None, None, None),
    'Other': (None, None, None, None)
}

# Convert to DataFrame
mapping_df = pd.DataFrame.from_dict(mapping, orient='index', 
    columns=['icd10_from', 'icd10_to', 'icd9_from', 'icd9_to'])
mapping_df.index.name = 'condition_label'
mapping_df = mapping_df.reset_index()

# Example: df already has a 'condition_label' column
# Merge the two DataFrames
df3 = df2.merge(mapping_df, on='condition_label', how='left')


In [80]:
df3.drop(columns=['icd_code_range_from', 'icd_code_range_to'], inplace=True)

In [81]:
df3.to_csv('NIES_2024_processed.csv', index=False)

# ICD Mapping ends

In [40]:
# ICD columns
ICD_cols = [f"Q45_QO{i}" for i in range(1, 16)]  # Q45_Q01 to Q45_Q015

In [55]:
df[ICD_cols]

,Q45_QO1,Q45_QO2,Q45_QO3,Q45_QO4,Q45_QO5,Q45_QO6,Q45_QO7,Q45_QO8,Q45_QO9,Q45_QO10,Q45_QO11,Q45_QO12,Q45_QO13,Q45_QO14,Q45_QO15
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
5,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12349,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12357,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN
12364,NaN,NaN,NaN,4.0,NaN,NaN,7.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,15.0


In [ ]:
'Q45_QO1' - 'Tumour or cancer'
'Q45_QO2' - 'Heart condition'
 'Q45_QO3' - 'Lung condition'
 'Q45_QO4' - 'Neurological condition'
 'Q45_QO5' - 'Orthopaedic condition'
 'Q45_QO6' - 'COVID 19'
 'Q45_QO7' - 'Infection (other than COVID 19)'
 'Q45_QO8' - 'Digestive system condition'
 'Q45_QO9' - 'Diabetes and related problems'
 'Q45_QO10'- 'Adverse reaction or poising'
 'Q45_QO11' - 'Injury and or accident'
 'Q45_QO12'- 'Mental health issue'
 'Q45_QO13' - 'Tests and or investigations'
 'Q45_QO14' - 'Dont know or wasnt told'
 'Q45_QO15' - 'Other'

,Q45_QO1,Q45_QO2,Q45_QO3,Q45_QO4,Q45_QO5,Q45_QO6,Q45_QO7,Q45_QO8,Q45_QO9,Q45_QO10,Q45_QO11,Q45_QO12,Q45_QO13,Q45_QO14,Q45_QO15
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
5,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12349,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12357,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN
12364,NaN,NaN,NaN,4.0,NaN,NaN,7.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,15.0


In [71]:
df

,Response_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q13,Q14,Q15,...,ScoreWAIT,ScoreOVERALL,treatment_score,pain_score,explanation_score,outcome_proxy,satisfaction_score,condition_label,icd_code_range_from,icd_code_range_to
1,2,1.0,1.0,4.0,2.0,2.0,6.0,0.0,0.5,0.5,...,NaN,3.0,0.375000,0.5,0.50,0.3,0.395000,Diabetes and related problems,E08,E13
2,3,1.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,NaN,10.0,0.958333,1.0,1.00,1.0,0.991667,Orthopaedic condition,M00,M99
4,5,2.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,NaN,9.0,0.875000,1.0,1.00,0.9,0.935000,Other,None,None
5,6,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,...,3.0,10.0,0.958333,1.0,1.00,1.0,0.991667,Heart condition,I00,I99
8,9,2.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,NaN,10.0,0.916667,1.0,1.00,1.0,0.983333,Tumour or cancer,C00,D49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12349,12350,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,...,9.0,8.0,0.958333,1.0,1.00,0.8,0.911667,Infection (other than COVID 19),A00,B99
12357,12358,1.0,2.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,NaN,10.0,0.916667,1.0,0.75,1.0,0.933333,Lung condition,J00,J99
12363,12364,1.0,1.0,1.0,2.0,2.0,2.0,0.5,1.0,1.0,...,9.0,10.0,0.791667,1.0,1.00,1.0,0.958333,Injury and/or accident,S00,T88
12364,12365,1.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,...,18.0,10.0,0.958333,1.0,1.00,1.0,0.991667,Other,None,None


In [72]:
df.drop(columns=['condition_label','icd_code_range_from','icd_code_range_to'], inplace=True)

In [74]:
df3 = df.reset_index().melt(id_vars=df.index.name, 
                                          var_name='condition_code', 
                                          value_name='score')


In [75]:
df3

,condition_code,score
0,index,1
1,index,2
2,index,4
3,index,5
4,index,8
...,...,...
539683,satisfaction_score,0.911667
539684,satisfaction_score,0.933333
539685,satisfaction_score,0.958333
539686,satisfaction_score,0.991667


In [73]:
condition_labels = {
    'Q45_QO1': 'Tumour or cancer',
    'Q45_QO2': 'Heart condition',
    'Q45_QO3': 'Lung condition',
    'Q45_QO4': 'Neurological condition',
    'Q45_QO5': 'Orthopaedic condition',
    'Q45_QO6': 'COVID 19',
    'Q45_QO7': 'Infection (other than COVID 19)',
    'Q45_QO8': 'Digestive system condition',
    'Q45_QO9': 'Diabetes and related problems',
    'Q45_QO10': 'Adverse reaction or poisoning',
    'Q45_QO11': 'Injury and/or accident',
    'Q45_QO12': 'Mental health issue',
    'Q45_QO13': 'Tests and/or investigations',
    'Q45_QO14': 'Don’t know or wasn’t told',
    'Q45_QO15': 'Other'
}

df['condition_label'] = df['condition_code'].map(condition_labels)


KeyError: 'condition_code'

In [58]:
condition_mapping = {
    'Q45_QO1':  ('Tumour or cancer', 'C00', 'D49'),
    'Q45_QO2':  ('Heart condition', 'I00', 'I99'),
    'Q45_QO3':  ('Lung condition', 'J00', 'J99'),
    'Q45_QO4':  ('Neurological condition', 'G00', 'G99'),
    'Q45_QO5':  ('Orthopaedic condition', 'M00', 'M99'),
    'Q45_QO6':  ('COVID 19', 'U07', 'U07'),
    'Q45_QO7':  ('Infection (other than COVID 19)', 'A00', 'B99'),
    'Q45_QO8':  ('Digestive system condition', 'K00', 'K95'),
    'Q45_QO9':  ('Diabetes and related problems', 'E08', 'E13'),
    'Q45_QO10': ('Adverse reaction or poisoning', 'T36', 'T50'),
    'Q45_QO11': ('Injury and/or accident', 'S00', 'T88'),
    'Q45_QO12': ('Mental health issue', 'F00', 'F99'),
    'Q45_QO13': ('Tests and/or investigations', 'Z01', 'Z01'),
    'Q45_QO14': ('Don’t know or wasn’t told', None, None),
    'Q45_QO15': ('Other', None, None)
}

In [59]:
# for col in df.columns:
#     label, range_from, range_to = condition_mapping[col]
#     q45_matrix[f'{col}_label'] = label
#     q45_matrix[f'{col}_range_from'] = range_from
#     q45_matrix[f'{col}_range_to'] = range_to


for col, (label, range_from, range_to) in condition_mapping.items():
    condition_mask = df[col].notna()
    df.loc[condition_mask, 'condition_label'] = label
    df.loc[condition_mask, 'icd_code_range_from'] = range_from
    df.loc[condition_mask, 'icd_code_range_to'] = range_to

In [63]:
# df.drop(columns=ICD_cols, inplace=True)  # Drop original ICD columns

In [67]:
df2 = df[['agegroup', 'Gender', 'sex', 'disability', 'ethnicgrp', 'WaitTimeCats', 'HealthRegion','Source_of_Admission_Category', 'insure','medcard', 'HospSize', 'HospModel', 'admtype', 'admcode', 'xadmtype', 'AdmTypeBinary', 'ScoreOVERALL','satisfaction_score','condition_label','icd_code_range_from','icd_code_range_to']]

In [76]:
df2.head()

,agegroup,Gender,sex,disability,ethnicgrp,WaitTimeCats,HealthRegion,Source_of_Admission_Category,insure,medcard,...,HospModel,admtype,admcode,xadmtype,AdmTypeBinary,ScoreOVERALL,satisfaction_score,condition_label,icd_code_range_from,icd_code_range_to
1,3,1,1,0,0.0,NaN,1,1,1.0,0.0,...,2.0,1,1,1,0.0,3.0,0.395000,Diabetes and related problems,E08,E13
2,2,2,2,0,0.0,NaN,1,2,0.0,1.0,...,2.0,1,2,1,0.0,10.0,0.991667,Orthopaedic condition,M00,M99
4,1,2,2,0,0.0,NaN,1,1,1.0,1.0,...,2.0,2,1,2,1.0,9.0,0.935000,Other,None,None
5,2,1,1,1,0.0,1.0,1,1,0.0,0.0,...,2.0,1,1,1,0.0,10.0,0.991667,Heart condition,I00,I99
8,3,1,1,1,0.0,NaN,1,1,0.0,1.0,...,2.0,2,1,2,1.0,10.0,0.983333,Tumour or cancer,C00,D49


In [68]:
df2.to_csv('NIES_2024_processed.csv', index=False)

In [ ]:
 condition_mapping = {
    'Q45_QO1':  ('Tumour or cancer', 'C00–D49'),
    'Q45_QO2':  ('Heart condition', 'I00–I99'),
    'Q45_QO3':  ('Lung condition', 'J00–J99'),
    'Q45_QO4':  ('Neurological condition', 'G00–G99'),
    'Q45_QO5':  ('Orthopaedic condition', 'M00–M99'),
    'Q45_QO6':  ('COVID 19', 'U07.1'),
    'Q45_QO7':  ('Infection (other than COVID 19)', 'A00–B99'),
    'Q45_QO8':  ('Digestive system condition', 'K00–K95'),
    'Q45_QO9':  ('Diabetes and related problems', 'E08–E13'),
    'Q45_QO10': ('Adverse reaction or poisoning', 'T36–T50'),
    'Q45_QO11': ('Injury and/or accident', 'S00–T88'),
    'Q45_QO12': ('Mental health issue', 'F00–F99'),
    'Q45_QO13': ('Tests and/or investigations', 'Z01'),
    'Q45_QO14': ('Don’t know or wasn’t told', None),
    'Q45_QO15': ('Other', None)
}


In [46]:
import numpy as np

# Create empty columns
df['condition_label'] = np.nan
df['icd_code_range'] = np.nan

# Loop through columns to populate new fields
for col, (label, icd_range) in condition_mapping.items():
    condition_mask = df[col].notna()
    df.loc[condition_mask, 'condition_label'] = label
    df.loc[condition_mask, 'icd_code_range'] = icd_range


C:\Users\Harishankar\AppData\Local\Temp\ipykernel_33740\1216347762.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tumour or cancer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[condition_mask, 'condition_label'] = label
C:\Users\Harishankar\AppData\Local\Temp\ipykernel_33740\1216347762.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'C00–D49' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[condition_mask, 'icd_code_range'] = icd_range


In [ ]:
# df.drop(columns=['condition_label','icd_code_range'], inplace=True)  # Drop original ICD columns

In [49]:
category_map = {
    'Q45_QO1':  ('Tumour or cancer', 'C00', 'D49'),
    'Q45_QO2':  ('Heart condition', 'I00', 'I99'),
    'Q45_QO3':  ('Lung condition', 'J00', 'J99'),
    'Q45_QO4':  ('Neurological condition', 'G00', 'G99'),
    'Q45_QO5':  ('Orthopaedic condition', 'M00', 'M99'),
    'Q45_QO6':  ('COVID 19', 'U07', 'U07'),
    'Q45_QO7':  ('Infection (other than COVID 19)', 'A00', 'B99'),
    'Q45_QO8':  ('Digestive system condition', 'K00', 'K95'),
    'Q45_QO9':  ('Diabetes and related problems', 'E08', 'E13'),
    'Q45_QO10': ('Adverse reaction or poisoning', 'T36', 'T50'),
    'Q45_QO11': ('Injury and/or accident', 'S00', 'T88'),
    'Q45_QO12': ('Mental health issue', 'F00', 'F99'),
    'Q45_QO13': ('Tests and/or investigations', 'Z01', 'Z01'),
    'Q45_QO14': ('Don’t know or wasn’t told', None, None),
    'Q45_QO15': ('Other', None, None)
}
# Convert ICD code to prefix for comparison (first 3 characters)
def normalize_icd(icd):
    return str(icd).replace('.', '')[:3]



# Preprocess mapping to usable format (convert range like 'C00–D49' to start and end)
parsed_mapping = {}
for key, (label, code_range) in condition_mapping.items():
    if code_range and '–' in code_range:
        start, end = code_range.split('–')
    elif code_range:
        start = end = code_range.replace('.', '')[:3]
    else:
        start = end = None
    parsed_mapping[key] = (label, start, end)


In [50]:

# Apply mapping
def map_condition(icd):
    icd_prefix = normalize_icd(icd)
    for key, (label, start, end) in parsed_mapping.items():
        if start and end and start <= icd_prefix <= end:
            return pd.Series([label, f"{start}–{end}"])
    return pd.Series(['Other', None])

# Apply to DataFrame
df[['condition_label', 'icd_code_range']] = df['icd_code'].apply(map_condition)

KeyError: 'icd_code'

In [52]:
df2[['condition_label', 'icd_code_range']] = df2['icd_code'].apply(map_condition)

In [54]:
df2['condition_label'].unique()

array(['Heart condition', 'Other', 'Digestive system condition',
       'Injury and/or accident', 'Lung condition',
       'Infection (other than COVID 19)', 'Neurological condition',
       'Mental health issue'], dtype=object)

In [51]:
import pandas as pd

# Raw data as list of tuples
data = [
    ("684f68ef68f898b125636a53", 11126859, 21137424, 1, "I25110", 10),
    ("684f68ef68f898b125636a54", 11126859, 21137424, 2, "I9589", 10),
    ("684f68ef68f898b125636a55", 11126859, 21137424, 3, "I10", 10),
    ("684f68ef68f898b125636a56", 11126859, 21137424, 4, "Z006", 10),
    ("684f68ef68f898b125636a57", 11126859, 21137424, 5, "K219", 10),
    ("684f68ef68f898b125636a58", 11126859, 21137424, 6, "E785", 10),
    ("684f68ef68f898b125636a59", 11126859, 21137424, 7, "Z8546", 10),
    ("684f68ef68f898b125636a5a", 11126859, 21137424, 8, "Z7982", 10),
    ("684f68f168f898b125636a60", 14568429, 29172242, 1, "S2241XA", 10),
    ("684f68f168f898b125636a61", 14568429, 29172242, 2, "R6521", 10),
    ("684f68f168f898b125636a62", 14568429, 29172242, 3, "I4901", 10),
    ("684f68f168f898b125636a63", 14568429, 29172242, 4, "J690", 10),
    ("684f68f168f898b125636a64", 14568429, 29172242, 5, "A419", 10),
    ("684f68f168f898b125636a65", 14568429, 29172242, 6, "G9340", 10),
    ("684f68f168f898b125636a66", 14568429, 29172242, 7, "I5023", 10),
    ("684f68f168f898b125636a67", 14568429, 29172242, 8, "N179", 10),
    ("684f68f168f898b125636a68", 14568429, 29172242, 9, "J9691", 10),
    ("684f68f168f898b125636a69", 14568429, 29172242, 10, "K831", 10),
    ("684f68f168f898b125636a6a", 14568429, 29172242, 11, "N186", 10),
    ("684f68f168f898b125636a6b", 14568429, 29172242, 12, "J95851", 10),
    ("684f68f168f898b125636a6c", 14568429, 29172242, 13, "E46", 10),
    ("684f68f168f898b125636a6d", 14568429, 29172242, 14, "E872", 10),
    ("684f68f168f898b125636a6e", 14568429, 29172242, 15, "L97419", 10),
    ("684f68f168f898b125636a6f", 14568429, 29172242, 16, "K567", 10),
    ("684f68f168f898b125636a70", 14568429, 29172242, 17, "F05", 10),
    ("684f68f168f898b125636a71", 14568429, 29172242, 18, "R188", 10),
    ("684f68f168f898b125636a72", 14568429, 29172242, 19, "D689", 10),
    ("684f68f168f898b125636a73", 14568429, 29172242, 20, "I120", 10),
    ("684f68f168f898b125636a74", 14568429, 29172242, 21, "I462", 10),
    ("684f68f168f898b125636a75", 14568429, 29172242, 22, "R1011", 10),
    ("684f68f168f898b125636a76", 14568429, 29172242, 23, "D696", 10),
    ("684f68f168f898b125636a77", 14568429, 29172242, 24, "E785", 10),
    ("684f68f168f898b125636a78", 14568429, 29172242, 25, "Z781", 10),
    ("684f68f168f898b125636a79", 14568429, 29172242, 26, "I2510", 10),
    ("684f68f168f898b125636a7a", 14568429, 29172242, 27, "Z955", 10),
    ("684f68f168f898b125636a7b", 14568429, 29172242, 28, "R4181", 10),
    ("684f68f168f898b125636a7c", 14568429, 29172242, 29, "Z7901", 10),
    ("684f68f168f898b125636a7d", 14568429, 29172242, 30, "W19XXXA", 10),
    ("684f68f168f898b125636a7e", 14568429, 29172242, 31, "Y92199", 10)
]

# Define column names
columns = ['_id', 'subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version']

# Create DataFrame
df2 = pd.DataFrame(data, columns=columns)

# View head of dataframe
print(df2.head())


                        _id  subject_id   hadm_id  seq_num icd_code  \
0  684f68ef68f898b125636a53    11126859  21137424        1   I25110   
1  684f68ef68f898b125636a54    11126859  21137424        2    I9589   
2  684f68ef68f898b125636a55    11126859  21137424        3      I10   
3  684f68ef68f898b125636a56    11126859  21137424        4     Z006   
4  684f68ef68f898b125636a57    11126859  21137424        5     K219   

   icd_version  
0           10  
1           10  
2           10  
3           10  
4           10  
